In [29]:
%load_ext autoreload
%autoreload 2

from FP567_Lib import *

import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import pathlib
import os
import glob
import json
import numpy as np
import pandas as pd

# improve/change plot appearance
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
'''
We have all the files in resources/market_item_data/, which are item infos in the form of:
{ 
    item_id : 
    [
        [date/time_n+1, price_at_time_n+1, amount_sold_at_time_n+1], [date/time_n+2, price_at_time_n+2, amount_sold_at_time_n+2], ..., [date/time_n+m, price_at_time_n+m, amount_sold_at_time_n+m]
    ]
}
for some amount of time.
The amount of time varies between items, as not all items have existed as long as others.

Let's make a Market object, which computes a bunch of the info we want.
'''
market = Market()

In [30]:
'''
So, lets extend all the items that are not highest_unit_time worth of info, backwards in time,
so that the items that do not have as many as highest_unit_time, now have highest_unit_time
amount of info, with the time stamp, but just 0, 0 for those days.

To do that, we can call the balance method of the Market object, giving it the 
longest time span of unix times, and 0, 0 as the default amount sold and price
'''
market.balance_as_is(0, 0)
market.is_balanced()

True

In [36]:
'''
Now we have a balanced market and want to include updates into a matrix in the below form.
Notice how it is essentially m matrices, where each matrix represents a unit of time,
appended onto one another from left to right, and is n rows by k+2 cols
        | unix_time_0                                                                                                       | unix_time_1                                                                                                       |     | unix_time_m 
________| amount_sold_0 | price_0 | update_unix_time_0_feat_1 | update_unix_time_0_feat_2 | ... | update_unix_time_0_feat_k | amount_sold_1 | price_1 | update_unix_time_1_feat_1 | update_unix_time_1_feat_2 | ... | update_unix_time_1_feat_k | ... | amount_sold_m | price_m | update_unix_time_m_feat_1 | update_unix_time_m_feat_2 | ... | update_unix_time_m_feat_k |
item_1  |               |         |                           |                           | ... |                           |               |         |                           |                           | ... |                           | ... |               |         |                           |                           | ... |                           |
item_2  |               |         |                           |                           | ... |                           |               |         |                           |                           | ... |                           | ... |               |         |                           |                           | ... |                           |
.       |       .           .                   .                       .                   ...               .                    .             .                .                            .                ...               .                           .           .                  .                           .                ...               .             | 
.       |       .           .                   .                       .                   ...               .                    .             .                .                            .                ...               .                           .           .                  .                           .                ...               .             | 
.       |       .           .                   .                       .                   ...               .                    .             .                .                            .                ...               .                           .           .                  .                           .                ...               .             | 
item_n  |               |         |                           |                           | ... |                           |               |         |                           |                           | ... |                           |     |               |         |                           |                           | ... |                           |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Alot of them will be zeros.
There will be zeros for
    update_unix_time_i_feat_j for all j when there was no update for unix_time_i.
    an item's amount_sold_i and price_i when that item was not being sold for unix_time_i.
'''


'\nNow we have a balanced market and want to include updates into a matrix in this form:\n        | unix_time_0                                                                                                    | unix_time_1                                                                                                    | unix_time_m \n________| amount_sold_0 | price_0 | update_unix_time_0_feat1 | update_unix_time_0_feat2 | ... | update_unix_time_0_featk | amount_sold_1 | price_1 | update_unix_time_1_feat1 | update_unix_time_1_feat2 | ... | update_unix_time_1_featk | ... | amount_sold_m | price_m | update_unix_time_m_feat1 | update_unix_time_m_feat2 | ... | update_unix_time_m_featk |\nitem_1  |               |         |                          |                          | ... |                          |               |         |                          |                          | ... |                          \nitem_2  |\n.\n.\n.\nitem_n  |\n'